In [4]:
import sys
print(sys.version)

3.7.0 (v3.7.0:1bf9cc5093, Jun 26 2018, 23:26:24) 
[Clang 6.0 (clang-600.0.57)]


In [5]:
# Not necessary at all, but to demonstrate that I'm aware that BeautifulSoup4 must be installed
!{sys.executable} -m pip install --upgrade pip
!{sys.executable} -m pip install BeautifulSoup4

Requirement already up-to-date: pip in /Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages (18.1)


In [103]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import json
import csv
import time
import datetime

In [151]:
Events = []
base_url = 'https://en.wikipedia.org'
main_url = base_url + '/wiki/List_of_UFC_events'

In [105]:
def perform_http_get(url):
    """
    Note:
        This is a function that tries to connect to a given URL, if it fails, a retry will occurs in 3 sec.  

    Args:
        url (str): A string representing the URL we want to integrate.

    Returns:
        The http response content 
    """
    r = requests.get(url)
    if r.status_code == 200:
        return BeautifulSoup(r.content, 'html.parser')
        

In [137]:
def extract_cell(cells, id_td):
    return cells[id_td].renderContents().decode().strip()

In [144]:
def append_fighter_names(cells_event, info):
    fighter1 = ''
    fighter2 = ''
    if len(cells_event[1].findAll('a')) == 0:
        fighter1 = extract_cell(cells_event, 1)
    else:
        fighter1 = cells_event[1].find('a').renderContents().decode().strip()

    if len(cells_event[3].findAll('a')) == 0:
        fighter2 = extract_cell(cells_event, 3)
    else:
        fighter2 = cells_event[3].find('a').renderContents().decode().strip()
    
    info.update({"fighter_1" : fighter1})
    info.update({"fighter_2" : fighter2})

In [145]:
def extract_row(cells_event, link):
       
    info = {
        "event_name": link.contents[0],
        "weight_class": extract_cell(cells_event, 0),
        "action": extract_cell(cells_event, 2),
        "method": extract_cell(cells_event, 4),
        "round": extract_cell(cells_event, 5),
        "time": extract_cell(cells_event, 6)
    }
    
    append_fighter_names(cells_event, info)
    Events.append(info)

In [116]:
def extract_info_individual_event(link):
    
    individual_event = perform_http_get(base_url + link.get('href'))
    table = individual_event.find('table',{'class': 'toccolours'})
    if table is not None:
        rows_event = table.findAll('tr')

        for row_event in rows_event:
            cells_event = row_event.findAll('td')
            if len(cells_event) > 0 :
                extract_row(cells_event, link)
                        

In [152]:
Events = []
soup = perform_http_get(main_url)
table_past_events = soup.find('table', {'id': 'Past_events'})

rows = table_past_events.findAll('tr')

for row in rows:
    cells = row.findAll('td')
    if len(cells) > 0 :
        links = cells[1].findAll('a')
        for link in links:        
            extract_info_individual_event(link)

{'event_name': 'UFC 229: Khabib vs. McGregor', 'weight_class': 'Lightweight', 'action': 'def.', 'method': 'Submission (neck crank)', 'round': '4', 'time': '3:03', 'fighter_1': 'Khabib Nurmagomedov', 'fighter_2': 'Conor McGregor'}
info2:  {'event_name': 'UFC 229: Khabib vs. McGregor', 'weight_class': 'Lightweight', 'action': 'def.', 'method': 'Submission (neck crank)', 'round': '4', 'time': '3:03', 'fighter_1': 'Khabib Nurmagomedov', 'fighter_2': 'Conor McGregor'}
{'event_name': 'UFC 229: Khabib vs. McGregor', 'weight_class': 'Lightweight', 'action': 'def.', 'method': 'TKO (corner stoppage)', 'round': '2', 'time': '5:00', 'fighter_1': 'Tony Ferguson', 'fighter_2': 'Anthony Pettis'}
info2:  {'event_name': 'UFC 229: Khabib vs. McGregor', 'weight_class': 'Lightweight', 'action': 'def.', 'method': 'TKO (corner stoppage)', 'round': '2', 'time': '5:00', 'fighter_1': 'Tony Ferguson', 'fighter_2': 'Anthony Pettis'}
{'event_name': 'UFC 229: Khabib vs. McGregor', 'weight_class': 'Light Heavyweigh

{'event_name': 'UFC Fight Night: Hunt vs. Oliynyk', 'weight_class': 'Heavyweight', 'action': 'def.', 'method': 'Submission (rear-naked choke)', 'round': '1', 'time': '4:26', 'fighter_1': 'Alexey Oleinik', 'fighter_2': 'Mark Hunt'}
info2:  {'event_name': 'UFC Fight Night: Hunt vs. Oliynyk', 'weight_class': 'Heavyweight', 'action': 'def.', 'method': 'Submission (rear-naked choke)', 'round': '1', 'time': '4:26', 'fighter_1': 'Alexey Oleinik', 'fighter_2': 'Mark Hunt'}
{'event_name': 'UFC Fight Night: Hunt vs. Oliynyk', 'weight_class': 'Light Heavyweight', 'action': 'def.', 'method': 'Submission (arm-triangle choke)', 'round': '2', 'time': '2:41', 'fighter_1': 'Jan Błachowicz', 'fighter_2': 'Nikita Krylov'}
info2:  {'event_name': 'UFC Fight Night: Hunt vs. Oliynyk', 'weight_class': 'Light Heavyweight', 'action': 'def.', 'method': 'Submission (arm-triangle choke)', 'round': '2', 'time': '2:41', 'fighter_1': 'Jan Błachowicz', 'fighter_2': 'Nikita Krylov'}
{'event_name': 'UFC Fight Night: Hun

{'event_name': 'UFC Fight Night: Gaethje vs. Vick', 'weight_class': 'Lightweight', 'action': 'def.', 'method': 'KO (punch)', 'round': '1', 'time': '1:27', 'fighter_1': 'Justin Gaethje', 'fighter_2': 'James Vick'}
info2:  {'event_name': 'UFC Fight Night: Gaethje vs. Vick', 'weight_class': 'Lightweight', 'action': 'def.', 'method': 'KO (punch)', 'round': '1', 'time': '1:27', 'fighter_1': 'Justin Gaethje', 'fighter_2': 'James Vick'}
{'event_name': 'UFC Fight Night: Gaethje vs. Vick', 'weight_class': 'Featherweight', 'action': 'def.', 'method': 'Decision (split) (29-28, 27-30, 29-28)', 'round': '3', 'time': '5:00', 'fighter_1': 'Michael Johnson', 'fighter_2': 'Andre Fili'}
info2:  {'event_name': 'UFC Fight Night: Gaethje vs. Vick', 'weight_class': 'Featherweight', 'action': 'def.', 'method': 'Decision (split) (29-28, 27-30, 29-28)', 'round': '3', 'time': '5:00', 'fighter_1': 'Michael Johnson', 'fighter_2': 'Andre Fili'}
{'event_name': 'UFC Fight Night: Gaethje vs. Vick', 'weight_class': "W

{'event_name': 'UFC on Fox: Alvarez vs. Poirier 2', 'weight_class': 'Lightweight', 'action': 'def.', 'method': 'TKO (punches and elbow)', 'round': '2', 'time': '4:05', 'fighter_1': 'Dustin Poirier', 'fighter_2': 'Eddie Alvarez'}
info2:  {'event_name': 'UFC on Fox: Alvarez vs. Poirier 2', 'weight_class': 'Lightweight', 'action': 'def.', 'method': 'TKO (punches and elbow)', 'round': '2', 'time': '4:05', 'fighter_1': 'Dustin Poirier', 'fighter_2': 'Eddie Alvarez'}
{'event_name': 'UFC on Fox: Alvarez vs. Poirier 2', 'weight_class': 'Featherweight', 'action': 'def.', 'method': 'TKO (punches)', 'round': '1', 'time': '4:19', 'fighter_1': 'José Aldo', 'fighter_2': 'Jeremy Stephens'}
info2:  {'event_name': 'UFC on Fox: Alvarez vs. Poirier 2', 'weight_class': 'Featherweight', 'action': 'def.', 'method': 'TKO (punches)', 'round': '1', 'time': '4:19', 'fighter_1': 'José Aldo', 'fighter_2': 'Jeremy Stephens'}
{'event_name': 'UFC on Fox: Alvarez vs. Poirier 2', 'weight_class': "Women's Strawweight",

{'event_name': 'UFC Fight Night: dos Santos vs. Ivanov', 'weight_class': 'Heavyweight', 'action': 'def.', 'method': 'Decision (unanimous) (50-45, 50-45, 50-45)', 'round': '5', 'time': '5:00', 'fighter_1': 'Junior dos Santos', 'fighter_2': 'Blagoy Ivanov'}
info2:  {'event_name': 'UFC Fight Night: dos Santos vs. Ivanov', 'weight_class': 'Heavyweight', 'action': 'def.', 'method': 'Decision (unanimous) (50-45, 50-45, 50-45)', 'round': '5', 'time': '5:00', 'fighter_1': 'Junior dos Santos', 'fighter_2': 'Blagoy Ivanov'}
{'event_name': 'UFC Fight Night: dos Santos vs. Ivanov', 'weight_class': 'Welterweight', 'action': 'def.', 'method': 'KO (punches)', 'round': '2', 'time': '3:13', 'fighter_1': 'Sage Northcutt', 'fighter_2': 'Zak Ottow'}
info2:  {'event_name': 'UFC Fight Night: dos Santos vs. Ivanov', 'weight_class': 'Welterweight', 'action': 'def.', 'method': 'KO (punches)', 'round': '2', 'time': '3:13', 'fighter_1': 'Sage Northcutt', 'fighter_2': 'Zak Ottow'}
{'event_name': 'UFC Fight Night:

...

{'event_name': 'UFC Fight Night: Thompson vs. Till', 'weight_class': 'Catchweight (174.5 lbs)', 'action': 'def.', 'method': 'Decision (unanimous) (48-47, 49-46, 49-46)', 'round': '5', 'time': '5:00', 'fighter_1': 'Darren Till', 'fighter_2': 'Stephen Thompson'}
info2:  {'event_name': 'UFC Fight Night: Thompson vs. Till', 'weight_class': 'Catchweight (174.5 lbs)', 'action': 'def.', 'method': 'Decision (unanimous) (48-47, 49-46, 49-46)', 'round': '5', 'time': '5:00', 'fighter_1': 'Darren Till', 'fighter_2': 'Stephen Thompson'}
{'event_name': 'UFC Fight Night: Thompson vs. Till', 'weight_class': 'Welterweight', 'action': 'def.', 'method': 'TKO (knee and punches)', 'round': '1', 'time': '4:32', 'fighter_1': 'Neil Magny', 'fighter_2': 'Craig White'}
info2:  {'event_name': 'UFC Fight Night: Thompson vs. Till', 'weight_class': 'Welterweight', 'action': 'def.', 'method': 'TKO (knee and punches)', 'round': '1', 'time': '4:32', 'fighter_1': 'Neil Magny', 'fighter_2': 'Craig White'}
{'event_name':

{'event_name': 'UFC 224: Nunes vs. Pennington', 'weight_class': "Women's Bantamweight", 'action': 'def.', 'method': 'TKO (punches)', 'round': '5', 'time': '2:36', 'fighter_1': 'Amanda Nunes', 'fighter_2': 'Raquel Pennington'}
info2:  {'event_name': 'UFC 224: Nunes vs. Pennington', 'weight_class': "Women's Bantamweight", 'action': 'def.', 'method': 'TKO (punches)', 'round': '5', 'time': '2:36', 'fighter_1': 'Amanda Nunes', 'fighter_2': 'Raquel Pennington'}
{'event_name': 'UFC 224: Nunes vs. Pennington', 'weight_class': 'Middleweight', 'action': 'def.', 'method': 'Decision (split) (29-28, 28-29, 29-28)', 'round': '3', 'time': '5:00', 'fighter_1': 'Kelvin Gastelum', 'fighter_2': 'Ronaldo Souza'}
info2:  {'event_name': 'UFC 224: Nunes vs. Pennington', 'weight_class': 'Middleweight', 'action': 'def.', 'method': 'Decision (split) (29-28, 28-29, 29-28)', 'round': '3', 'time': '5:00', 'fighter_1': 'Kelvin Gastelum', 'fighter_2': 'Ronaldo Souza'}
{'event_name': 'UFC 224: Nunes vs. Pennington', 

{'event_name': 'UFC on Fox: Poirier vs. Gaethje', 'weight_class': 'Lightweight', 'action': 'def.', 'method': 'TKO (punches)', 'round': '4', 'time': '0:33', 'fighter_1': 'Dustin Poirier', 'fighter_2': 'Justin Gaethje'}
info2:  {'event_name': 'UFC on Fox: Poirier vs. Gaethje', 'weight_class': 'Lightweight', 'action': 'def.', 'method': 'TKO (punches)', 'round': '4', 'time': '0:33', 'fighter_1': 'Dustin Poirier', 'fighter_2': 'Justin Gaethje'}
{'event_name': 'UFC on Fox: Poirier vs. Gaethje', 'weight_class': 'Welterweight', 'action': 'def.', 'method': 'Submission (guillotine choke)', 'round': '2', 'time': '3:17', 'fighter_1': 'Alex Oliveira', 'fighter_2': 'Carlos Condit'}
info2:  {'event_name': 'UFC on Fox: Poirier vs. Gaethje', 'weight_class': 'Welterweight', 'action': 'def.', 'method': 'Submission (guillotine choke)', 'round': '2', 'time': '3:17', 'fighter_1': 'Alex Oliveira', 'fighter_2': 'Carlos Condit'}
{'event_name': 'UFC on Fox: Poirier vs. Gaethje', 'weight_class': 'Middleweight', 

{'event_name': 'UFC Fight Night: Werdum vs. Volkov', 'weight_class': 'Heavyweight', 'action': 'def.', 'method': 'KO (punches)', 'round': '4', 'time': '1:38', 'fighter_1': 'Alexander Volkov', 'fighter_2': 'Fabrício Werdum'}
info2:  {'event_name': 'UFC Fight Night: Werdum vs. Volkov', 'weight_class': 'Heavyweight', 'action': 'def.', 'method': 'KO (punches)', 'round': '4', 'time': '1:38', 'fighter_1': 'Alexander Volkov', 'fighter_2': 'Fabrício Werdum'}
{'event_name': 'UFC Fight Night: Werdum vs. Volkov', 'weight_class': 'Light Heavyweight', 'action': 'def.', 'method': 'Decision (unanimous) (29-28, 29-28, 30-27)', 'round': '3', 'time': '5:00', 'fighter_1': 'Jan Błachowicz', 'fighter_2': 'Jimi Manuwa'}
info2:  {'event_name': 'UFC Fight Night: Werdum vs. Volkov', 'weight_class': 'Light Heavyweight', 'action': 'def.', 'method': 'Decision (unanimous) (29-28, 29-28, 30-27)', 'round': '3', 'time': '5:00', 'fighter_1': 'Jan Błachowicz', 'fighter_2': 'Jimi Manuwa'}
{'event_name': 'UFC Fight Night:

KeyboardInterrupt: 

In [153]:
print(len(Events))

df = pd.DataFrame(Events)
df = df[['event_name', 'weight_class', 'fighter_1', 'action', 'fighter_2', 'round', 'time', 'method' ]]

df


257


,action,event_name,fighter_1,fighter_2,method,round,time,weight_class
0,def.,UFC 229: Khabib vs. McGregor,Khabib Nurmagomedov,Conor McGregor,Submission (neck crank),4,3:03,Lightweight
1,def.,UFC 229: Khabib vs. McGregor,Tony Ferguson,Anthony Pettis,TKO (corner stoppage),2,5:00,Lightweight
2,def.,UFC 229: Khabib vs. McGregor,Dominick Reyes,Ovince Saint Preux,"Decision (unanimous) (30-27, 30-27, 30-27)",3,5:00,Light Heavyweight
3,def.,UFC 229: Khabib vs. McGregor,Derrick Lewis,Alexander Volkov,KO (punches),3,4:49,Heavyweight
4,def.,UFC 229: Khabib vs. McGregor,Michelle Waterson,Felice Herrig,"Decision (unanimous) (30-26, 29-28, 30-27)",3,5:00,Women's Strawweight
5,def.,UFC 229: Khabib vs. McGregor,Jussier Formiga,Sergio Pettis,"Decision (unanimous) (30-26, 29-28, 29-28)",3,5:00,Flyweight
6,def.,UFC 229: Khabib vs. McGregor,Vicente Luque,Jalin Turner,KO (punches),1,3:52,Welterweight
7,def.,UFC 229: Khabib vs. McGregor,Aspen Ladd,Tonya Evinger,TKO (punches),1,3:26,Women's Bantamweight
8,def.,UFC 229: Khabib vs. McGregor,Scott Holtzman,Alan Patrick,KO (elbows),3,3:42,Lightweight
9,def.,UFC 229: Khabib vs. McGregor,Yana Kunitskaya,Lina Länsberg,"Decision (unanimous) (30-27, 30-27, 30-27)",3,5:00,Women's Bantamweight
